In [1]:
import re
import sqlite3
from urllib.request import urlopen
from html import unescape
import pandas as pd
import os

In [ ]:
def main():
    html = fetch('http://www.hanbit.co.kr/store/books/full_book_list_html')
    books=scrape(html)
    save('books.db',books)

In [ ]:
def fetch(url):
    f = urlopen(url)
    encoding = f.info().get_content_charset(failobj="utf-8")
    html = f.read().decode(encoding)
    return html

In [ ]:
def scrape(html):
    books=[]
    for partial_html in re.findall(r'<td class="left"><a.*?></td>',html,re.DOTALL):
        url = re.search(r'<a href="(.*?)">',partial_html).group(1)
        url='http://hanbit.co.kr'+url
        title = re.sub(r'<.*?>','',partial_html)
        title = unescape(title)
        books.append(pd.DataFrame({'url':[url],'title':[title]}))
    return pd.concat(books)

In [ ]:
def save(db_path, books):
    with sqlite3.connect(os.path.join('.', db_path)) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            books.to_sql(name = 'BOOKS_INFO', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
    
    query = 'SELECT * FROM BOOKS_INFO'
    df = pd.read_sql(query, con = con)
    return df

In [ ]:
html = fetch('http://www.hanbit.co.kr/store/books/full_book_list.html')

In [ ]:
df = scrape(html)
df.reset_index(drop=True, inplace=True)
df2 = save('books.db', df)
df2

In [4]:
!pip install lxml
!pip install cssselect

In [5]:
import lxml.html

In [7]:
for a in html.cssselect('a'):
    print(a.get('href'),a.text)

NameError: name 'html' is not defined